<a href="https://colab.research.google.com/github/omarinho1230/quora_duplicate_question/blob/master/quora_duplicate_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **import data from kaggle**

In [0]:
!pip install kaggle


In [0]:
!mkdir .kaggle

In [0]:
!ls

In [0]:
import json
token = {"username":"username","key":"key"}
with open('/content/.kaggle/kaggle.json', 'w') as file:json.dump(token, file)
!cp '/content/.kaggle/kaggle.json' ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d quora/question-pairs-dataset
!unzip \*.zip


In [0]:
!ls

### **Dataset preparation and preprocessing**

In [0]:
import pandas as pd
import numpy as np

data = pd.read_csv('questions.csv') #dataframe///
data = data.drop(['id', 'qid1', 'qid2'], axis=1)
data.head()


In [0]:
data=data[:20000]
data

In [0]:
data.describe()

In [0]:
data[data.isna().any(axis=1)]

In [0]:
data=data.dropna()
data = data.reset_index(drop=True)

In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_special(text):
    special_chars = "[~#$%^@&*&()+-_\",?!.:[]\\;><`|{}=\/\'=»¿シし]"
    for k in special_chars:
            if type(text) == str:
                if k=="-" or k=="_":
                    text=text.replace(k, "")
                else:
                    text=text.replace(k, " ")       
    return text

def contractions(sent):
    sub_pattern = [("will not","won't"),("shall not","shan't"),
                (" not", "n\'t"),(" will","\'ll"),(" is","\'s"),
                   (" am","\'m"),(" are","\'re"),(" is","who\'s")]
    sent2=sent.split(" ")
    hold = ""
    for k in range(len(sent2)):
        kk = sent2[k]
        for rep in range(len(sub_pattern)):
            kk = re.sub(sub_pattern[rep][1],sub_pattern[rep][0],kk)
        hold = hold + " " + kk
    return hold.lower()

def remove_stop_words(text):
    
    stops1 = [word.lower() for word in stop_words]
    punctuation = [',','.','!','?',';','-']
    hold = []
    if type(text) == list:
        for word in range(len(text)):
            if text[word].lower() in stops1 or text[word].lower() in punctuation or text[word].lower() == "xxxxxx":
                continue
            else:
                hold.append(text[word].lower())
    return hold

def wordTok(sent):
    tok = word_tokenize(sent)
    return tok

def pipeline(text):
    text = contractions(text)
    text = remove_special(text)
    textToks = wordTok(text)
    textToks = remove_stop_words(textToks)
    final = ""
    for k in range(len(textToks)):
            final = final+textToks[k]+" "
    return final.strip().lower()



In [0]:
 import nltk
 nltk.download('punkt')
data["question1"] = list(map(pipeline,data["question1"]))
data["question2"] = list(map(pipeline,data["question2"]))
data.head()

### **length based features**

In [0]:


data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q1'].head()



In [0]:
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
data['len_q2'].head()

### **difference in lengths of two questions**


In [0]:
# 
data['diff_len'] = data.len_q1 - data.len_q2
data['diff_len'].head()

### **character length based features**

In [0]:
# 
data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q1'].head()


In [0]:
data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q2'].head()

### **word length based features**

In [0]:

data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
data['len_word_q1']

In [0]:
data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))
data['len_word_q2']


In [0]:
# common words in the two questions
data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split())
.intersection(set(str(x['question2']).lower().split()))), axis=1)
data['common_words'].head()

In [0]:
fs_1 = ['len_q1', 'len_q2', 'diff_len', 'len_char_q1', 
        'len_char_q2', 'len_word_q1', 'len_word_q2',     
        'common_words']   #list


number of words we have in this dataset is:

In [0]:
from nltk import word_tokenize
allWords = ""
for k in range (len(data["question1"])):
    allWords=allWords+data["question1"][k].strip().lower()+" "
    allWords=allWords+data["question2"][k].strip().lower()+" "
allcount = len(set(word_tokenize(allWords)))
print(allcount)

**features of similarity**

In [0]:
!pip3 install fuzzywuzzy[speedup]
from fuzzywuzzy import fuzz


**compares the entire question similarity**

In [0]:
data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_qratio'].head()

**compares partial question similarity**

In [0]:
data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_ratio'].head()

**Wratio to eliminate lower and upper cases and more**




In [0]:
data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_WRatio'].head()

**partial token set ratio**

In [0]:

data['fuzz_partial_token_set_ratio'] = data.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)

data['fuzz_partial_token_set_ratio'].head()

In [0]:
data['fuzz_partial_token_sort_ratio'] = data.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_token_sort_ratio'].head()

In [0]:

data['fuzz_token_set_ratio'] = data.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_set_ratio'].head()

In [0]:

data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_sort_ratio'].head()

In [0]:


fs_2 = ['fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio', 
       'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
       'fuzz_token_set_ratio', 'fuzz_token_sort_ratio']  #list



**import word2vec pre-trained Google News corpus word vector model**

In [0]:
!brew install wget

!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

**load the model**

In [0]:
from gensim import models
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
model =models.KeyedVectors.load_word2vec_format(
    EMBEDDING_FILE, binary=True)


In [0]:
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

**Word mover distance**

In [0]:
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

In [0]:
data['wmd'] = data.apply(lambda x: wmd(data['question1'], data['question2']), axis=1)

**transform words to vectors**

In [0]:
def sent2vec(s, model): 
    M = []
    words = str(s).lower().split()
    for word in words:
      if word not in stop_words:
        if word.isalpha():
          if word in model:
            M.append(model[word])
    M = np.array(M)
    if len(M) > 0:
        v = M.sum(axis=0)
        return v / np.sqrt((v ** 2).sum())
    else:
        return np.zeros(300)  #1*300
        

In [0]:
w2v_q1 = np.array([sent2vec(q, model) for q in data.question1])
w2v_q2 = np.array([sent2vec(q, model) for q in data.question2])


**padding the vectors**

In [0]:
from keras.preprocessing.sequence import pad_sequences
STRING_SIZE= max(data.question1.map(lambda x: len(x)).max(),
                     data.question2.map(lambda x: len(x)).max())
sent1pad = pad_sequences(w2v_q1, maxlen=STRING_SIZE,padding='post',truncating='post',dtype='float32')
sent2pad = pad_sequences(w2v_q2,maxlen=STRING_SIZE,padding='post',truncating='post',dtype='float32')
sent1pad

**cosine distance**

In [0]:
from scipy.spatial.distance import cosine

data['cosine_distance'] = [cosine(x,y) for (x,y) in zip(w2v_q1, w2v_q2)]  #x y 1-D array

fs4_1 = ['cosine_distance','wmd']

**scale the data**

In [0]:


from sklearn import linear_model
from sklearn.preprocessing import StandardScaler


In [0]:
scaler = StandardScaler()
y = data.is_duplicate.values  #ndarray


In [0]:
y = y.astype('float32').reshape(-1, 1)  #ndarray



In [0]:
Y=pd.DataFrame(data=y)


In [0]:
X = data.copy().drop(["question1", "question2","is_duplicate"], axis=1)
X

In [0]:
X = X.replace([np.inf, -np.inf], np.nan).fillna(0).values  #ndarray
X = scaler.fit_transform(X)  #ndarray


**split the data**

In [0]:

from sklearn.model_selection import train_test_split
np.random.seed(0)  #same results will be generated although shuffle is used
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


**logistic regression model**


In [0]:
logres = linear_model.LogisticRegression(C=0.1, 
                                 solver='sag', max_iter=5000)
                               
logres.fit(x_train, y_train.ravel())

lr_preds = logres.predict(x_test)

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [0]:

cm = confusion_matrix(y_test, lr_preds)

cm

In [0]:
print("logistic regression accuracy: "+str(accuracy_score(lr_preds, y_test)))

**some other models**

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [0]:
LinearDiscriminantAnalysis_model=LinearDiscriminantAnalysis()
KNeighborsClassifier_model=KNeighborsClassifier()
GaussianNB_model=GaussianNB()
DecisionTreeClassifier_model=DecisionTreeClassifier()
SVC_model=SVC()

In [0]:

KNeighborsClassifier_model.fit(x_train, y_train.ravel())
GaussianNB_model.fit(x_train, y_train.ravel())
DecisionTreeClassifier_model.fit(x_train, y_train.ravel())
SVC_model.fit(x_train, y_train.ravel())

In [0]:
nb_preds = GaussianNB_model.predict(x_test)
knn_preds = KNeighborsClassifier_model.predict(x_test)
dtc_preds = DecisionTreeClassifier_model.predict(x_test)
svc_preds = SVC_model.predict(x_test)



In [0]:
# Accuracy score is the simplest way to evaluate
print("naive bayesian accuracy:"+str(accuracy_score(nb_preds, y_test)))
print("knn accuracy:"+str(accuracy_score(knn_preds, y_test)))
print("decision tree accuracy:"+str(accuracy_score(dtc_preds, y_test)))
print("support vector machines accuracy :"+str(accuracy_score(svc_preds, y_test)))
# But Confusion Matrix and Classification Report give more details about performance
print("===================knn===========================================")
print(classification_report(knn_preds, y_test))
print("===================naive bayesian================================")
print(classification_report(nb_preds, y_test))
print("===================decision tree=================================")
print(classification_report(dtc_preds, y_test))
print("===================support vector machines=======================")
print(classification_report(svc_preds, y_test))

**artificial neural network**

In [0]:
print(x_train.shape)
print(y_train.shape)


In [0]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [0]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(17, activation='relu', kernel_initializer='random_normal', input_dim=17))
#Second  Hidden Layer
classifier.add(Dense(100, activation='tanh', kernel_initializer='random_normal'))
#Third  Hidden Layer
classifier.add(Dense(100, activation='tanh', kernel_initializer='random_normal'))
#Fourth  Hidden Layer
classifier.add(Dense(100, activation='relu', kernel_initializer='random_normal'))
#Fifth  Hidden Layer
classifier.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [0]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['acc'])

In [0]:
from tensorflow.keras.utils import plot_model
plot_model(classifier, to_file='model.png')

In [0]:
#Fitting the data to the training dataset
modelann=classifier.fit(x_train, y_train.ravel(),verbose=True, batch_size=10, epochs=50 ,validation_data=(x_test, y_test.ravel()))

In [0]:
eval_model=classifier.evaluate(x_test, y_test.ravel())
eval_model

In [0]:
y_pred=classifier.predict(x_test)
y_pred =(y_pred>0.5)

In [0]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [0]:
classifier.summary()

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(modelann.history['acc'])
plt.plot(modelann.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(modelann.history['loss'])
plt.plot(modelann.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

**prepare the data to siamese lstm model**

In [0]:


split=int(0.8*len(sent1pad))

y = data["is_duplicate"]
x1_train = sent1pad[:split]
x1_val = sent2pad[split:]
x2_train = sent2pad[:split]
x2_val = sent1pad[split:]
y_train = y[:split]
y_val = y[split:]




x1_train = np.asarray(x1_train,dtype='float32')
x1_val = np.asarray(x1_val,dtype='float32')
x2_train = np.asarray(x2_train,dtype='float32')
x2_val = np.asarray(x2_val,dtype='float32')
y_train = np.asarray(y_train,dtype='float32')
y_val = np.asarray(y_val,dtype='float32')

TRAIN_SIZE = len(x1_train)
VAL_SIZE = len(x1_val)

In [0]:
print(x1_train.shape)
print(x1_val.shape)
print(x2_train.shape)
print(x2_val.shape)
print(y_train.shape)
print(y_val.shape)



In [0]:

EMBEDDING_SIZE=256
x1_train


In [0]:
y_train

In [0]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [0]:
x2_train.shape

In [0]:
from time import time
import datetime
from keras.layers import Dense,LSTM,Input,Activation,Flatten,Concatenate,Reshape,Embedding,Bidirectional,Dropout, Lambda,BatchNormalization
from keras.models import Model,Sequential
import keras.backend as K
from keras.optimizers import Adadelta
gradient_clipping_norm = 1.25
model1_input = Input(shape=(x1_train.shape[1],),name="input1")
model1_embedding = Embedding(allcount,output_dim=EMBEDDING_SIZE,input_length=STRING_SIZE,name="embedding1")(model1_input)
model1_LSTM = LSTM(256,return_sequences=False,activation='relu',dropout=0.4,name="LSTM1")(model1_embedding)


model2_input = Input(shape=(x2_train.shape[1],),name='input2')
model2_embedding = Embedding(allcount,output_dim=EMBEDDING_SIZE,input_length=STRING_SIZE,name="embedding2")(model2_input)
model2_LSTM = LSTM(256,return_sequences=False,activation='relu',dropout=0.4,name="LSTM2")(model2_embedding)


# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([model1_LSTM, model2_LSTM])
normalization=BatchNormalization()(malstm_distance)
before_last_layer=Dense(1, activation='tanh', kernel_initializer='glorot_uniform',name="before_output")(normalization)
normalization2=BatchNormalization()(before_last_layer)
output=Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform',name="output")(normalization2)
 #Pack it all up into a model
malstm = Model([model1_input, model2_input], [output])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([x1_train, x2_train], y_train, batch_size=100, nb_epoch=10,
                            validation_data=([x1_val, x2_val], y_val))

print("Training time finished.\n{} epochs in {}".format(30, datetime.timedelta(seconds=time()-training_start_time)))
malstm.summary()

In [0]:

malstm.evaluate([x1_val,x2_val],y_val)

In [0]:
from keras.utils import plot_model
plot_model(malstm, to_file='model.png')

# New Section

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(malstm_trained.history['accuracy'])
plt.plot(malstm_trained.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()